In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv()
llm=GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
print(llm("Write few lines on GenAI"))

C:\Users\sresa\AppData\Local\Temp\ipykernel_20412\474359270.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("Write few lines on GenAI"))


GenAI, or Generative AI, refers to artificial intelligence models that can generate new content, such as text, images, audio, and video. These models learn patterns from existing data and then use that knowledge to create original outputs.  From writing poems and code to designing products and composing music, GenAI is rapidly transforming various industries with its creative potential and automation capabilities. However, ethical considerations regarding copyright, bias, and misinformation are crucial as GenAI becomes more prevalent.


In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "retails"  # MySQL database name

db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
)

print(db.table_info)



CREATE TABLE sales (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from sales table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [5]:
from langchain.chains import create_sql_query_chain
ch=create_sql_query_chain(llm,db)
res=ch.invoke({"question":"How many customers are there"})
res

'```sql\nSELECT count(DISTINCT `CustomerID`) FROM sales\n```'

In [6]:
cleaned = res.replace("```sql\n", "").replace("\n```", "")
print(cleaned)

SELECT count(DISTINCT `CustomerID`) FROM sales


In [7]:
result=db.run(cleaned)
print(result)

[(29,)]


In [8]:
ch=create_sql_query_chain(llm,db)
def execute_query(question):
    try:
        # Generate SQL query from question
        res=ch.invoke({"question":question})
        print(res)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned = res.replace("```sql\n", "").replace("\n```", "")
        print(cleaned)
        print("###################################################")        
        # Execute the cleaned query
        result=db.run(cleaned)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")

In [9]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

```sql
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` FROM sales GROUP BY `ProductCategory`
```
###################################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` FROM sales GROUP BY `ProductCategory`
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [10]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount`
FROM sales
GROUP BY `ProductCategory`
ORDER BY `TotalSalesAmount` DESC;
```
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount`
FROM sales
GROUP BY `ProductCategory`
ORDER BY `TotalSalesAmount` DESC;
###################################################
###################################################
[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]


In [11]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

```sql
SELECT 
    `Gender`,
    AVG(`Age`) AS `AverageAge`
FROM 
    `sales`
GROUP BY 
    `Gender`
```
###################################################
SELECT 
    `Gender`,
    AVG(`Age`) AS `AverageAge`
FROM 
    `sales`
GROUP BY 
    `Gender`
###################################################
###################################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [12]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

```sql
SELECT
  `CustomerID`,
  SUM(`TotalAmount`) AS `TotalSpent`
FROM sales
GROUP BY
  `CustomerID`
ORDER BY
  `TotalSpent` DESC
LIMIT 5;
```
###################################################
SELECT
  `CustomerID`,
  SUM(`TotalAmount`) AS `TotalSpent`
FROM sales
GROUP BY
  `CustomerID`
ORDER BY
  `TotalSpent` DESC
LIMIT 5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [13]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

```sql
SELECT
  DATE_FORMAT(`Date`, '%Y-%m') AS `Month`,
  COUNT(*) AS `TransactionCount`
FROM sales
GROUP BY
  `Month`
ORDER BY
  `Month`
LIMIT 5;
```
###################################################
SELECT
  DATE_FORMAT(`Date`, '%Y-%m') AS `Month`,
  COUNT(*) AS `TransactionCount`
FROM sales
GROUP BY
  `Month`
ORDER BY
  `Month`
LIMIT 5;
###################################################
###################################################
[('2023-01', 4), ('2023-02', 4), ('2023-03', 1), ('2023-04', 5), ('2023-05', 2)]


In [14]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

```sql
SELECT 
    `ProductCategory`,
    SUM(`TotalAmount`) AS `TotalSalesAmount`,
    AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM sales
GROUP BY `ProductCategory`
```
###################################################
SELECT 
    `ProductCategory`,
    SUM(`TotalAmount`) AS `TotalSalesAmount`,
    AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM sales
GROUP BY `ProductCategory`
###################################################
###################################################
[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]
